# Scenario C - Peak Number Variation (multiple runs)

In this scenario the number of peaks in a generated dataset is varied from low to high, 
the rest of the parameters is kept constant (noise level = 1%). The number of peaks expected by the probabilistic model is varied between the low and high peak number.

The model used in the inference of the parameters is formulated as follows:

\begin{equation}
\large y = f(x) = \sum\limits_{m=1}^M \big[A_m \cdot e^{-\frac{(x-\mu_m)^2}{2\cdot\sigma_m^2}}\big] + \epsilon
\end{equation}

This file runs a series of inference runs for a set of generated spectra. New spectra are generated for each run and stored. After running inference, only the summary statistics are stored and the next run is started.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import arviz as az

#az.style.use('arviz-darkgrid')

print('Running on PyMC3 v{}'.format(pm.__version__))

## Import local modules

In [ ]:
import os
import sys
sys.path.append('../../modules')
import datagen as dg
import models as mdl
import results as res
import figures as fig
import settings as cnf

## Local configuration

In [ ]:
# output for results and images
out_path      = './output_mruns_5x5'
file_basename = out_path + '/scenario_peaks'
        
# if dir does not exist, create it
if not os.path.exists(out_path):
    os.makedirs(out_path)

conf = {}
    
# scenario name
conf['scenario'] = 'peak variation'
    
# initialization method for sampler
conf['init_mode'] = 'adapt_diag'

# probabilistic model (priors)
conf['prior_model'] = 'lognormal'

# provide peak positions to the model as testvalues ('yes'/'no')
conf['peak_info'] = 'yes'

# data mode ('generate'/'preload')
conf['data_mode'] = 'generate'

# number of runs
conf['nruns'] = 4

# number of cores to run sampling chains on
conf['ncores'] = 2

# number of samples per chain
conf['nsamples'] = 2000

In [ ]:
conf

## Save configuration

In [ ]:
cnf.save(out_path, conf)

# Generate data and plot

In [ ]:
# list of wavelengths (x-values)
xval = [i for i in range(200, 400, 2)]

# number of spectra per peak number
nsets  = 4

# number of peaks in the spectrum
peak_numbers = [2,3,4,5,6]

# total number of datasets
tsets = nsets * len(peak_numbers)

# total number of inference runs (per run)
truns = nsets * len(peak_numbers)**2

# generate nruns sets of spectra
for r in range(conf['nruns']):
    print("Generating dataset {0} of {1}".format(r+1,conf['nruns']))
    
    ldata, lpeaks, lpeakdata  = [], [], []
    
    # create output directory for data
    out_dir = out_path + '/run_{0:02d}'.format(r+1)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
            
    for pn in peak_numbers:
        for i in range(nsets):
            df, peaks, df_peakinfo = dg.data_generator(xvalues=xval, nsamples=15, npeaks=pn)
            ldata.append(df)
            lpeaks.append(peaks)
            lpeakdata.append(df_peakinfo)
            
    # save data and peak information to disk
    for i in range(len(ldata)):
        ldata[i].to_csv(out_dir + '/dataset_{0:02d}.csv'.format(i+1), index=False)
        lpeakdata[i].to_csv(out_dir + '/peakdata_{0:02d}.csv'.format(i+1), index=False)
    dg.data_save(out_dir + '/peakinfo.csv', lpeaks)
    
    # plot datasets
    filen = out_dir + '/scenario_peaks'
    fig.plot_datasets(ldata, lpeaks, dims=(int(tsets/2),2), figure_size=(12,int(tsets*(1.8))), 
                                                        savefig='yes', fname=filen)

In [ ]:
print("total number of multiple runs                    : {0}".format(conf['nruns']))
print("total number of peak numbers                     : {0}".format(len(peak_numbers)))
print("total number of datasets per peak number         : {0}".format(nsets))
print("total number of datasets per model               : {0}".format(tsets))
print("total number of inference runs (per single loop) : {0}".format(truns))

# Load data, run inference, visualize, collect results and save 

In [ ]:
# convert pandas data to numpy arrays
x_val = np.array(xval, dtype='float32')

In [ ]:
# dataframe to hold multiple run results
res_df = pd.DataFrame()

# run the whole loop of inference, posterior sampling, results collection and saving
for r in range(conf['nruns']):
    print("starting loop {0}/{1}".format(r+1,conf['nruns']))

    models, traces, lmodpeak = [], [], []

    # load datasets from disk
    data_dir = out_path + '/run_{0:02d}'.format(r+1)
    ldata, lpeaks, lpeakdata = dg.data_load(tsets, data_dir)

    # store dataset y-values in list
    cols = ldata[0].columns
    y_val = [ldata[i][cols].values for i in range(len(ldata))]

    # actual inference run number
    inf_run = 1

    for pn in peak_numbers:
        print("running {0}-peak model".format(pn))
        for i in range(len(ldata)):
            if conf['peak_info'] == 'yes':
                # Get the peak numbers from the list. If the actual peak number in the spectrum is 
                # lower than what the model is expecting, then expand the list to the expected size,
                # duplicating the existing peak mu values, else truncate the list (taking the peaks
                # with the highest amplitude).
                plist = sorted(lpeaks[i])
                if len(plist) < pn:
                    pl = sorted(np.resize(plist, (1,pn)).flatten())
                else:
                    # sort peak info dataframe on amplitude value 
                    l1 = lpeakdata[i].sort_values('amp', ascending=False)
                    # truncate list to expected peak number
                    pl = l1['mu'].values[:pn]

                model_g = mdl.model_gauss(xvalues=x_val, observations=y_val[i], npeaks=pn, 
                                          mu_peaks=pl, pmodel=conf['prior_model'])
            else:
                model_g = mdl.model_gauss(xvalues=x_val, observations=y_val[i], npeaks=pn,
                                          pmodel=conf['prior_model'])      
            models.append(model_g)

            with model_g:
                print("({6}:{2}/{3}) running inference on dataset #{0}/{1} [{4}-peak model:{5}-peak spectrum]"
                      .format(i+1,len(ldata),inf_run,truns,pn,len(plist),r+1))
                lmodpeak += [(pn,len(plist))]
                trace_g = pm.sample(conf['nsamples'], init=conf['init_mode'], cores=conf['ncores'])
                traces.append(trace_g)
                inf_run += 1

    # save model figure as image (once)
    if r == 0:
        img = pm.model_to_graphviz(models[0])
        img.render(filename=file_basename + '_model', format='png');

    # posterior predictive traces
    ppc = [pm.sample_posterior_predictive(traces[i], samples=500, model=models[i]) for i in range(len(traces))]

    # collect the results, concat single run result to overall result 
    varnames = ['amp', 'mu', 'sigma', 'epsilon']
    lruns = ['{0}'.format(r+1) for i in range(truns)]
    df = res.get_results_summary(varnames, traces, ppc, y_val, epsilon_real=0.05, sets=tsets, 
                                 labels=lmodpeak, runlist=lruns)
    
    res_df = res_df.append(df, ignore_index=True)

## Show results and save

In [ ]:
res_df

In [ ]:
# save results to .csv
res_df.to_csv(out_path + '/scenario_peaks_mruns.csv', index=False)

In [ ]:
cnf.close(out_path)